The goal of this code is to remove leading and trailing silences of a list of 461 audio files by performing the following:
1. Remove any leading or trailing silences in the duration of an entire audio file, with the silence threshold set at -40dBFS i.e. any segments with loudness of -40dBFS or lower will be regarded as a silent portion
2. Creating a new excel sheet with the (1) duration 

In [1]:
import pandas as pd
import numpy as np
import os
from pydub import AudioSegment
from pydub import AudioSegment, silence
from os.path import join

/Users/jannaha/anaconda3/lib/python3.10/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
#Specify folder path where original audio files are stored 
audio_folder = '/Users/jen/OneDrive - National University of Singapore/Coding/Phase_2_Analysis/del_silence'

In [3]:
#Specify folder path where manipulated audio files will be stored
output_folder = '/Users/jen/OneDrive - National University of Singapore/Coding/Phase_2_Analysis/del_silence/del_silence_output'

In [4]:
#Path of excel file where list of audio file names (.wav) are stored
excel_file = '/Users/jen/OneDrive - National University of Singapore/Coding/Phase_2_Analysis/del_silence.xlsx'

In [5]:
df = pd.read_excel(excel_file)

In [6]:
df.head()

,wavefile,word,token_no
0,001saya03.wav,saya,3
1,002kita03.wav,kita,3
2,003kalau03.wav,kalau,3
3,004jadi03.wav,jadi,3
4,005tapi02.wav,tapi,2


In [8]:
#Number of rows corresponds to number of audio (.wav) files in folder are stored in 
df.shape

(461, 3)

In [9]:
#Creating a new dataframe to store metadata or attributes of each audio file 
#The duration of original audio file, timestamp where silence is at is added as columns in df2 dataframe
df2 = pd.DataFrame(columns = ['duration', 'silence', 'new_duration','new_RMS'])

In [10]:
def get_info():
    for i in df.index:
        audiofile = df.loc[i, 'wavefile']
        wavefile_path = os.path.join(audio_folder, audiofile)
        
        #check if wavefile path exists: ## out after using 
        ##print(os.path.isfile(wavefile_path))
        
        sound = AudioSegment.from_file(wavefile_path)
        
        #search for sections with silences, silence threshold set at -40dBFS
        silence_dur = silence.detect_silence(sound, min_silence_len= 50, silence_thresh=-50, seek_step =1)
        total_dur = len(sound)
        list_count = len(silence_dur)
        
        #if else statement to remove only leading and trailing silences rather than silences in between word utterance
        if list_count == 1:
            new_file = sound[:int(silence_dur[0][0])]
            new_file_path = output_folder + "/" + audiofile[:-4] + "-new.wav"
            new_file.export(new_file_path, format = "wav")
            new_sound = AudioSegment.from_file(new_file_path)
        elif list_count > 1:
            new_file = sound[int(silence_dur[0][1]):int(silence_dur[-1][0])]
            new_file_path = output_folder + "/" + audiofile[:-4] + "-new.wav"
            new_file.export(new_file_path, format = "wav")
            new_sound = AudioSegment.from_file(new_file_path)
        
        newfile_dur = len(new_sound)
        dbfs_loudness = new_sound.dBFS
        
        df2.loc[i] = [total_dur, silence_dur, new_file_dur, dbfs_loudness]


In [12]:
get_info()

In [13]:
df2.head()

,duration,silence
0,690,"[[526, 690]]"
1,703,"[[0, 68], [95, 161], [287, 379], [582, 703]]"
2,1024,"[[0, 124], [719, 807], [967, 1024]]"
3,670,"[[532, 649]]"
4,527,"[[0, 56], [185, 319], [398, 527]]"


In [14]:
#creating excel writer
write_to = pd.ExcelWriter('silence_output.xlsx')

#writing dataframe to excel sheet, named Sheet 1
df2.to_excel(write_to, 'Sheet1')

#save excel file
write_to.save()